In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Dec 18 06:00:06 2019

@author: satida
"""


from Satida.Layers import ReaderXML,Tokenizer,StopWord,Dictionary,Replacement,Compress,Posting,PostingCompress
from Satida.BTtree import BTree



class Sequential:
    
    
    def __init__(self):
        
        self.model=list()
        self.docs=None
        self.tokens=None
        self.tokens_sw=None
        self.dict=None
        self.rpc=None
        self.comp=None
        print("MODEL")

        
    
    def add(self,Layer):
        """
        add()
        ------
        
        Satida.Sequential.add (method)
        
        Params :
        ------
        Layer : Satida.Layers.object as input):\n
        
        return :
        ---------
        nothing
        """
        self.model.append(Layer)
    
    
    def fit(self):
        """
        via this method you can run your model.\n 
        this method dose not return any value as result.
        
        PrintSize :
        ------------
        
        after finished process save model in directory and print size of it .
        """
        for ly in self.model:
            if type(ly) == type(ReaderXML(["null"],["null"])):
                try :
                    self.docs,self.org_Docs=ly.execute()
                    print("ReaderXML Finished !")
                except:
                    raise Exception('there is a problem in Reading XML Files !')
            
            elif type(ly) == type(Tokenizer()):
                try:
                    self.tokens_docs=ly.execute(self.docs)
                    temp=list()
                    for doc in self.tokens_docs:
                        temp+=doc
                    self.tokens = temp[:]
                    del temp
                    print("Tokenizer Finished !")
                except:
                    raise Exception('there is a problem in Reading XML Files or Tokenizer !')
                    
            elif type(ly) == type(StopWord(["null"])):
                self.tokens_sw=ly.execute(self.tokens)
                self.stopword_tool=ly.execute
                print("StopWord Finished !")
            
            elif type(ly) == type(Dictionary()):
                if self.tokens_sw != None:
                    self.dict=ly.execute(self.tokens_sw)
                else :
                    self.dict=ly.execute(self.tokens)
                print("Dictionary Finished !")
            
            elif type(ly) == type(Replacement()):
                self.rpc=ly.execute(self.dict)
                print("Replacement Finished !")
            
            
            elif type(ly) == type(Posting()):
                self.rep_to_dict,self.posting=ly.execute(self.dict,self.rpc,self.tokens_docs)
                print("Posting Finished !")
            
            elif type(ly) == type(PostingCompress("Gama")):
                self.posting_compress,self.p_decompress=ly.execute(self.posting)
                print("PostingCompress Finished !")
            
            elif type(ly) == type(Compress("Full")):
                try:
                    if self.rpc != None:
                        self.comp=ly.execute(self.rpc)
                    else :
                        self.comp=ly.execute(self.dict)
                    print("Compress Finished !")
                    # pickle 
                    #with open("Normal.pkl","wb") as f:
                        #pickle.dump(self.comp,f)
                except:
                    raise Exception('there is a problem in Replacement or Dictionary !')
        
            elif type(ly) == BTree:
                self.tree=ly
                mode=ly.name
                if mode == "Simple":
                    ly.execute(None)
                    if self.rpc != None:
                        ly.Insert(self.rpc)
                    else :
                        ly.Insert(self.dict)
                    
                    ly.PrintSize()
                else :
                    ly.execute(self.comp["String"])
                    ly.Insert(self.comp["indexes"])
                    ly.PrintSize()
            
                
            
    def __intersection(self,lst):
      lst1= lst[0]
      for lst2 in lst[1:]:
        lst1 = [value for value in lst1 if value in lst2] 
       
      return lst1          
                

    def Search(self,items:str):
      useful_docs=list()
      for item in self.stopword_tool(items.split(" ")):
        result_str,refrence = self.__REDetector(item)
        dict_index = self.rep_to_dict[refrence[0]]
        if self.posting_compress:
          useful_docs.append(self.p_decompress(self.posting_compress)[dict_index])
          print("Posting decompressed !")
        else :
          useful_docs.append(self.posting[dict_index])
      result=self.__intersection(useful_docs)
      for doc in result:
        print(20*"*")
        self.__print(self.org_Docs[doc].find("TITLE").text)
        print(10*"*",20*" ",10*"*")
        self.__print(self.org_Docs[doc].find("TEXT").text)
        print(20*"*")
      return result_str,refrence

            
            
          
    
    
    def __REDetector(self,word:str):
        
        from Satida.Utils.Utils import alphabet
        import re
        
        refrences =list()
        result = list()
        alphabet="".join(alphabet).replace("\n","")
        
        if re.match(r"(\*["+alphabet+"]+\*)",word) != None:
            word=word.replace("*","")
            res_str,res_ref=self.tree.Search(word)
            for i,r in enumerate(res_str):
                if word in r:
                    result.append(r)
                    refrences.append(res_ref[i])
                    
        elif re.match(r"(["+alphabet+"]+\*["+alphabet+"]+)",word) != None:
            op1,op2=word.split("*")
            res_str,res_ref=self.tree.Search(op2+"$"+op1)
            for i,r in enumerate(res_str):
                if r.startswith(op1) and r.endswith(op2):
                    result.append(r)
                    refrences.append(res_ref[i])
            
        elif re.match(r"(["+alphabet+"]+\*)",word) != None:
            word=word.replace("*","")
            res_str,res_ref=self.tree.Search(word)
            for i,r in enumerate(res_str):
                if r.startswith(word) :
                    result.append(r)
                    refrences.append(res_ref[i])
        
        elif re.match(r"(\*["+alphabet+"]+)",word) != None:
            word=word.replace("*","")
            res_str,res_ref=self.tree.Search(word +"$")
            for i,r in enumerate(res_str):
                if r.endswith(word) :
                    result.append(r)
                    refrences.append(res_ref[i])
                    
        elif re.match(r"(["+alphabet+"]+)",word) != None:
            word=word.replace("*","")
            res_str,res_ref=self.tree.Search(word +"$")
            for i,r in enumerate(res_str):
                if r == word  :
                    result.append(r)
                    refrences.append(res_ref[i])
        
        return result,refrences
         
    
    
    def __print(self,text):
      for i in range(len(text) // 60 +1):
        try:
          print(text[i*60:(i+1)*60])
        except:
          print("sdfdfs")
          print(text[i*60:])
      
        

    
    
    
    
    
    
    
    
    